https://dev.twitter.com/rest/reference/get/statuses/lookup

Script takes list of tweet_id s and returns the specified information for the tweets and the user who owns the tweet. Script reads tweet_id s from "statuses_lookup_in.txt". Before running the script specify your delimeter: ',' (comma), ' ' (space) or '\n' (new line) as default.

user_show is used to retrieve a single user object.

There are a few things to note when using this method:

1. You need to enter your API credentials to the key list.
2. You can specify which info you need to the info list. You can find the name of the returned parameters (id, followers_count etc.) in the link above.
3. Script outputs the resulting data to user_lookup_out.txt as a csv formatted file

In [79]:
from birdy.twitter import UserClient, BirdyException 
from time import sleep
import string
import time

key = [
    ["Consumer Key", "Consumer Secret", "Access Token", "Access Token Secret"]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [83]:
delim = '\n'

f = open('statuses_lookup_in.txt', 'r', encoding='utf-8')
statusIDs = f.read().split(delim)
f.close()
    
print('Number of user IDs: ' + str(len(statusIDs)))

sInfo = [
    'retweet_count',
    'favorite_count'
]

uInfo = [
    'id',
    'screen_name',
    'description',
    'followers_count',
    'statuses_count'
]

Number of user IDs: 147861


In [ ]:
t0 = time.time()

statusInfo = []
keyInd = 0

for i in range(int(len(statusIDs)/100)+1):
    print(i)
    imin = i*100
    imax = (i+1)*100
    if i == int(len(statusIDs)/100):
        imax = len(statusIDs)
    while(True):
        try:
            response = client.api.statuses.lookup.get(id=statusIDs[imin : imax])
            for rd in response.data:
                tmpInfo = []
                for i in uInfo:
                    tmpInfo.append(rd['user'][i])
                for i in sInfo:
                    tmpInfo.append(rd[i])
                statusInfo.append(tmpInfo)
            break
        except Exception as err:
            print(err.status_code)
            print(err)
            if err.status_code == 429:
                sleep(60)
                keyInd = (keyInd + 1)%len(key)
            else:
                break
            client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
    
print(time.time() - t0)

In [76]:
statusDict = {}
userDict = {}

for si in statusInfo:
    if si[0] not in statusDict:
        userDict[si[0]] = si[:5]
        statusDict[si[0]] = si[5:] + [1]
    else:
        statusDict[si[0]][0] += si[5]
        statusDict[si[0]][1] += si[6]
        statusDict[si[0]][2] += 1

In [65]:
filename = 'statuses_lookup_desc_out.txt'
f = open(filename, 'w', encoding='utf-8')

for i in uInfo:
    f.write(str(i))
    f.write(',')
    
f.write(str(sInfo[0]))
for i in sInfo[1:]:
    f.write(',')
    f.write(str(i))
f.write(',coup_tweets')
f.write("\n")

for uid, userInfo in userDict.items():
    for ui in userInfo:
        f.write(str(ui))
        f.write(',')
    f.write(str(statusDict[uid][0]))
    for si in statusDict[uid][1:]:
        f.write(',')
        f.write(str(si))
    f.write("\n")
f.close()

In [78]:
#Without Description
filename = 'statuses_lookup_out.txt'
f = open(filename, 'w', encoding='utf-8')

descInd = uInfo.index('description')

for i in uInfo:
    if i == 'description':
        continue
    f.write(str(i))
    f.write(',')
    
f.write(str(sInfo[0]))
for i in sInfo[1:]:
    f.write(',')
    f.write(str(i))
f.write(',coup_tweets')
f.write("\n")

for uid, userInfo in userDict.items():
    for i, ui in enumerate(userInfo):
        if i == descInd:
            continue
        f.write(str(ui))
        f.write(',')
    f.write(str(statusDict[uid][0]))
    for si in statusDict[uid][1:]:
        f.write(',')
        f.write(str(si))
    f.write("\n")
f.close()